# FRR, SiteRM, XrootD on Fabric

This setup will install the following:
* 1 Router VM with FRR.
* 3 Hosts (XRootD) and all will point as GW to Router;
* SiteRM Frontend + Agent's will be installed on Router and Host VMs
* Monitoring node with Grafana/Prometheus and full ELStack.

In [ ]:
import os
import json
import traceback
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
import ipaddress

from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
import mflib 
from mflib.mflib import MFLib

fablib = fablib_manager()
                     
fablib.show_config();
slice_name = 'FRR-CERN'

# Check version
print(f"MFLib version  {mflib.__version__} " )


## Create the Experiment Slice

# Configuration parameters

In [3]:
site = 'CERN'
routername = 'frr-cern'
image_name = "docker_ubuntu_22"
host_image = 'docker_rocky_8'

# 2602:fcfb:001d:fff0::/64 - is used for Peering with Cisco Fabric;
frr_config = {'range': '2602:fcfb:001d:fff0::/64',
             'ip': '2602:fcfb:001d:fff0::2/64',
             'gw': '2602:fcfb:001d:fff0::1',
             'vlan': 99}

# 2602:fcfb:001d:fff1::64, 2602:fcfb:001d:fff2::/64, 2602:fcfb:001d:fff3::/64 
# Those IP Ranges uses for FRR node
ip_ranges = {"2602:fcfb:001d:fff1::/64": {
    'frr': "2602:fcfb:001d:fff1::a/64",
    'vlan': 100},
        "2602:fcfb:001d:fff2::/64": {
    'frr': "2602:fcfb:001d:fff2::a/64",
    'vlan': 101},
        "2602:fcfb:001d:fff3::/64": {
    'frr': "2602:fcfb:001d:fff3::a/64",
    'vlan': 102}}

# Host IPs and corresponding gateways (pointing to FRR)
hosts = {"host1": {
    '2602:fcfb:001d:fff1::2/64': '2602:fcfb:001d:fff1::a',
    '2602:fcfb:001d:fff2::2/64': '2602:fcfb:001d:fff2::a',
    '2602:fcfb:001d:fff3::2/64': '2602:fcfb:001d:fff3::a',
}, "host2": {
    '2602:fcfb:001d:fff1::3/64': '2602:fcfb:001d:fff1::a',
    '2602:fcfb:001d:fff2::3/64': '2602:fcfb:001d:fff2::a',
    '2602:fcfb:001d:fff3::3/64': '2602:fcfb:001d:fff3::a',
}, "host3": {
    '2602:fcfb:001d:fff1::4/64': '2602:fcfb:001d:fff1::a',
    '2602:fcfb:001d:fff2::4/64': '2602:fcfb:001d:fff2::a',
    '2602:fcfb:001d:fff3::4/64': '2602:fcfb:001d:fff3::a',
}}

hostspub = {"host1": {"2602:fcfb:001d:fff0::a/64": '2602:fcfb:001d:fff0::2/64'},
            "host2": {"2602:fcfb:001d:fff0::b/64": '2602:fcfb:001d:fff0::2/64'},
            "host3": {"2602:fcfb:001d:fff0::c/64": '2602:fcfb:001d:fff0::2/64'}}


# List of destinations, for which to add static routes and use dataplane interface
use_dataplane_routes = {"2602:fcfb::/36": "Fabric",
                        "2001:1458::/32": "Cern",
                        "2620:6a::/48": "Fermilab",
                        "2605:d9c0::/32": "Caltech",
                        "2600:900::/28": "Nebraska",
                        "2001:48d0::/32": "SDSC"}

# XRootd configurations
# Location of XRootD Redirector (only passed as env parameter to xrootd)
xrd_redir = "cern-sub1-host1-redir.exp.fabric-testbed.net"
# XRootD HTTP Requires to provide secret (and must be equal between all xrootd servers).
# Modify as you see it fit.
xrd_http_secret = "pYrySWhj4BftwJkSbMyAk8ha3p5YXsAt7g3mFzx7Vkg"




# Helper Functions
import subprocess
import shlex

def runcmd(nodes, commands):
    print("RUN COMMANDS")
    for command in commands:
        for nodename in nodes:
            node = slice.get_node(name=nodename)
            print(f'Execute: {command} on {nodename}')
            stdout, stderr = node.execute(command)

def runonecmd(node, nodename, command):
    print(f'Execute: {command} on {nodename}')
    stdout, stderr = node.execute(command)

def runIntfTuning(nodes):
    print("NODE INTERFACE TUNING")
    for nodename in nodes:
        node = slice.get_node(name=nodename)
        for intf in node.get_interfaces():
            command = f"sudo sh vpp-frr/interface-tuning.sh {intf.get_device_name()}"
            print(f'Execute: {command} on {nodename}')
            stdout, stderr = node.execute(command)

def localcmd(command):
    """Execute command locally and return stdout and stderr."""
    print(f'Execute local cmd: {command}')
    command = shlex.split(str(command))
    proc = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    return proc.communicate()





# Create Slice

In [ ]:
#Create Slice
slice = fablib.new_slice(name=slice_name)

# Add Router;
router = slice.add_node(name=routername, site=site, cores=16, ram=16, disk=10, image=image_name)
routernic = router.add_component(model='NIC_ConnectX_6', name='nic_local').get_interfaces()
# This will get dedicated NIC with 2 ports.
# Port-1 - Is used for Peering with Fabric Cisco
# Port-2 - Is used for hosts (L2 Network)
# For SENSE Needs - everything uses sub-interfaces;
subnet = IPv6Network(frr_config['range'])
vlan = frr_config['vlan']
net1 = slice.add_l3network(name=f'pub-0', type='IPv6Ext', subnet=subnet)
ciface = routernic[1].add_sub_interface(f"vlan{vlan}", vlan=str(vlan), bw=100)
ciface.set_mode('manual')
net1.add_interface(ciface)

# Add Networks and interfaces to FRR
counter=0
for iprange, rangeitems in ip_ranges.items():
    subnet = IPv6Network(iprange)
    vlan = rangeitems['vlan']
    gateway = subnet[1]
    net1 = slice.add_l2network(name=f'test-{counter}', subnet=subnet, gateway=gateway)
    ciface = routernic[0].add_sub_interface(f"vlan{vlan}", vlan=str(vlan), bw=100)
    ciface.set_mode('manual')
    net1.add_interface(ciface)        
    counter+=1

# Add Network and interfaces to Hosts
counter = 0
intcounter = 0
for host, rangeitems in hosts.items():
    node = slice.add_node(name=f"{host}", site=site, cores=4, ram=8, disk=100, image=host_image)
    counter = 0
    for hostip, hostgw in rangeitems.items():
        iface = node.add_component(model='NIC_Basic', name=f'nic_local-{counter}-{intcounter}').get_interfaces()[0]
        iface.set_mode('manual')
        net1 = slice.get_network(name=f'test-{counter}')
        net1.add_interface(iface)
        intcounter+=1
        counter+=1
    # Add Also pub-0 basic
    iface = node.add_component(model='NIC_Basic', name=f'nic_pub0-{counter}-{intcounter}').get_interfaces()[0]
    iface.set_mode('manual')
    net1 = slice.get_network(name=f'pub-0')
    net1.add_interface(iface)

# Add Measurement node:
MFLib.addMeasNode(slice, site=site, disk=100)

slice.submit()

## Deploy monitoring (Takes long time to install)

In [ ]:
print(f"MFLib version  {mflib.__version__} " )
# Import MFLib Class
from mflib.mflib import MFLib
# The slice name of the slice with which you want to interact.
mf = MFLib(slice_name, mf_repo_branch="main")
print(mf.grafana_tunnel)
instrumetize_results = mf.instrumentize()
print(mf.grafana_tunnel)

print(f"Browse to https://localhost:{mf.grafana_tunnel_local_port}/grafana/dashboards?query=%2A")

# All node tuning for high throughput

In [ ]:
commands = ["sudo apt update",
            "sudo apt install -y docker-compose-plugin", 
            "sudo usermod -aG docker ubuntu",
            "git clone https://github.com/sdn-sense/vpp-frr",
            "sudo sh vpp-frr/tuning.sh"]
runcmd([routername], commands)
print('-'*100)
runIntfTuning([routername])

# Install docker, clone tuning, and tune node;
commands = ["sudo service docker start",
           "git clone https://github.com/sdn-sense/vpp-frr",
            "sudo sh vpp-frr/tuning.sh"]
runcmd(list(hosts.keys()), commands)
print('-'*100)
runIntfTuning(list(hosts.keys()))

## Configure IPs on FRR and Hosts (set GWs)

In [ ]:
router = slice.get_node(name=routername)
# 1. Disable accept_ra for the first interface. That add a default route and we want to avoid that
router_iface = router.get_interface(network_name=f'pub-0')
iface = router_iface.get_physical_os_interface_name()
runonecmd(router, routername, f"sudo sysctl -w net.ipv6.conf.{iface.replace('.', '/')}.accept_ra=0")
runonecmd(router, routername, f"sudo ip link set {iface} down")
runonecmd(router, routername, f"sudo ip link set {iface} up")
# 2. Disable accept_ra also for vlan device
iface = router_iface.get_device_name()
runonecmd(router, routername, f"sudo sysctl -w net.ipv6.conf.{iface.replace('.', '/')}.accept_ra=0")
runonecmd(router, routername, f"sudo ip link set {iface} down")
runonecmd(router, routername, f"sudo ip link set {iface} up")
# 3. Set IP and default routes as defined in configuration
#   Issue is on Fabric IPv6 Mgmt hosts - default route will be via mgmt;
# Deleting it will restore back. Would need explicit routing, but at the same time
# need a way to control mgmt interface setup. Best would be if there is an option
# to say that mgmt interface - do not set default route and use only explicit routing.
# If we want to make it static - we also need to modify netplan (which is also controlled by fabric)
iface = router_iface.get_device_name()
command = f"sudo ip addr add {frr_config['ip']} dev {iface}"
runonecmd(router, routername, command)
# 4. Set default routes for all our known ranges (CERN,Fabric,Caltech,SDSC,Fermilab,Nebraska, etc..)
for iprange, name in use_dataplane_routes.items():
    command = f"sudo ip -6 route add {iprange} via {frr_config['gw']} dev {iface}"
    runonecmd(router, routername, command)

# For all other L2 networks for Hosts - Move interfaces on FRR also to Net-NS and set IPs;
counter=0
intfdone = []
for iprange, rangeitems in ip_ranges.items():
    router_iface = router.get_interface(network_name=f'test-{counter}')
    # Disable accept_ra
    iface = router_iface.get_physical_os_interface_name()
    if iface not in intfdone:
        runonecmd(router, routername, f"sudo sysctl -w net.ipv6.conf.{iface.replace('.', '/')}.accept_ra=0")
        runonecmd(router, routername, f"sudo ip link set {iface} down")
        runonecmd(router, routername, f"sudo ip link set {iface} up")
        intfdone.append(iface)
    # Disable accept_ra for sub device
    iface = router_iface.get_device_name()
    if iface not in intfdone:
        runonecmd(router, routername, f"sudo sysctl -w net.ipv6.conf.{iface.replace('.', '/')}.accept_ra=0")
        runonecmd(router, routername, f"sudo ip link set {iface} down")
        runonecmd(router, routername, f"sudo ip link set {iface} up")
        intfdone.append(iface)
    command = f"sudo ip addr add {rangeitems['frr']} dev {iface}"
    runonecmd(router, routername, command)
    counter+=1

# Set IPs on Hosts;
counter = 0
intcounter = 0
for host, rangeitems in hosts.items():
    node1 = slice.get_node(name=host)
    counter = 0
    intfdone = []
    for hostip, hostgw in rangeitems.items():
        node1_iface = node1.get_interface(name=f'{host}-nic_local-{counter}-{intcounter}-p1')        
        intf_name = node1_iface.get_device_name()
        net1 = slice.get_network(name=f'test-{counter}')
        if intf_name not in intfdone:
            runonecmd(node1, host, f"sudo sysctl -w net.ipv6.conf.{intf_name.replace('.', '/')}.accept_ra=0")
            runonecmd(node1, host, f"sudo ip link set {intf_name} down")
            runonecmd(node1, host, f"sudo ip link set {intf_name} up")
            intfdone.append(intf_name)
        command = f"sudo ip addr add {hostip} dev {intf_name}"
        runonecmd(node1, host, command)
        intcounter+=1
        counter+=1

In [ ]:
for host, hostips in hostspub.items():
    node1 = slice.get_node(name=host)
    counter = 0
    for ip, gw in hostips.items():
        node1_iface = node1.get_interface(network_name=f'pub-{counter}')        
        intf_name = node1_iface.get_device_name()
        runonecmd(node1, host, f"sudo sysctl -w net.ipv6.conf.{intf_name.replace('.', '/')}.accept_ra=0")
        runonecmd(node1, host, f"sudo ip link set {intf_name} down")
        runonecmd(node1, host, f"sudo ip link set {intf_name} up")
        runonecmd(node1, host, f"sudo ip addr add {ip} dev {intf_name}")
        for iprange, name in use_dataplane_routes.items():
            command = f"sudo ip -6 route add {iprange} via {gw.split('/')[0]} dev {intf_name}"
            runonecmd(node1, host, command)
        counter+=1

## Deploy FRR and configure FRR to be a router.

In [ ]:
node1 = slice.get_node(name=routername)
commands = ["sudo nft flush ruleset",
    "cd vpp-frr/frr-no-dpdk/ && sudo docker compose up -d",
    "sh vpp-frr/frr-no-dpdk/keygen.sh"]
runcmd([routername], commands)

## Download SiteRM Repo to all nodes

In [ ]:
slice = fablib.get_slice(name=slice_name)
commands = ["git clone https://github.com/sdn-sense/siterm-startup"]
node1 = slice.get_node(name=routername)
runcmd([routername], commands)
for host in list(hosts.keys()):
    runcmd([host], commands)

## Copy Certificates, creds from your bastion node

In [ ]:
commands = ["cd ~/vpp-frr && git pull"]
runcmd([routername], commands)
print('-'*100)

# Install docker, clone tuning, and tune node;
commands = ["cd ~/vpp-frr && git pull"]
runcmd(list(hosts.keys()), commands)
print('-'*100)

# Generate and print commands to copy cert's to required locations
import os

if not os.path.isfile('macaroon-secret'):
    print('If you dont have macaroon secret, generate it using the following command')
    print('openssl rand -base64 -out macaroon-secret 64')
    print('This file will need to be equal between all nodes')
    print('-'*80)

scp_cmd = "scp -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config "

# frr node for SiteRM FE
# hosts - for SiteRM Agent;
#         for XRootD;
slice = fablib.get_slice(name=slice_name)
router = slice.get_node(name=routername)
user = router.get_username()
mgmtip = router.get_management_ip()
for dest, srcfile in {'~/siterm-startup/fe/conf/etc/httpd/certs/cert.pem': 'hostcert.pem',
                      '~/siterm-startup/fe/conf/etc/grid-security/hostcert.pem': 'hostcert.pem',
                      '~/siterm-startup/fe/conf/etc/httpd/certs/privkey.pem': 'hostkey.pem',
                      '~/siterm-startup/fe/conf/etc/grid-security/hostkey.pem': 'hostkey.pem'}.items():
    cmd = f"{scp_cmd} {srcfile} {user}@[{mgmtip}]:{dest}"
    print(localcmd(cmd))
# Print host commands
for hostname in list(hosts.keys()):
    node = slice.get_node(name=hostname)
    user = node.get_username()
    mgmtip = node.get_management_ip()
    for dest, srcfile in {'~/siterm-startup/agent/conf/etc/grid-security/hostcert.pem': 'hostcert.pem',
                          '~/siterm-startup/agent/conf/etc/grid-security/hostkey.pem': 'hostkey.pem',
                          '~/vpp-frr/xrootd/priv/xrootdcert.pem': 'hostcert.pem',
                          '~/vpp-frr/xrootd/priv/xrootdkey.pem': 'hostkey.pem',
                          '~/vpp-frr/xrootd/priv/macaroon-secret': 'macaroon-secret'}.items():
        cmd = f"{scp_cmd} {srcfile} {user}@[{mgmtip}]:{dest}"
        print(localcmd(cmd))


## Deploy SiteRM Frontend

In [ ]:
commands = ["cd siterm-startup && git pull",
            f"cp vpp-frr/siterm/{site}/fe/ansible-conf.yaml siterm-startup/fe/conf/etc/ansible-conf.yaml",
            f"cp vpp-frr/siterm/{site}/fe/siterm.yaml siterm-startup/fe/conf/etc/siterm.yaml",
            f"cp vpp-frr/siterm/{site}/fe/environment siterm-startup/fe/conf/environment",
            f'cp ~/.ssh/frrsshkey siterm-startup/fe/conf/opt/siterm/config/ssh-keys/frrsshkey',
            "cd siterm-startup/fe/docker/ && ./run.sh -i dev -n host"]
node1 = slice.get_node(name=routername)
runcmd([routername], commands)

## Deploy SiteRM Agent(s)

In [ ]:
for host in list(hosts.keys()):
    commands = ["cd siterm-startup && git pull",
                f"cp vpp-frr/siterm/{site}/{host}/siterm.yaml siterm-startup/agent/conf/etc/siterm.yaml",
                 "cd siterm-startup/agent/docker/ && sudo ./run.sh -i dev"]
    runcmd([host], commands)

## Deploy XRootD on all Hosts

# Configure certs, users, dir and config file for xrootd;

In [ ]:
slice = fablib.get_slice(name=slice_name)
commands = ["cd ~/vpp-frr && git pull"]
runcmd(list(hosts.keys()), commands)
print('-'*100)


commands = ["sudo python3 ~/vpp-frr/xrootd/config/default/opt/usergroupchecker.py --notimeout",
            "sudo chown xrootd:xrootd ~/vpp-frr/xrootd/priv/*.pem",
            "sudo chmod 600 ~/vpp-frr/xrootd/priv/xrootdkey.pem",
            "sudo mkdir -p /storage/cms/store/",
            "sudo chown cmsuser:cmsuser /storage/cms/store/",
            "sed -i 's|XRD_HTTP_SECRET_KEY=.*|XRD_HTTP_SECRET_KEY=%s|' ~/vpp-frr/xrootd/.env" % xrd_http_secret,
            "sed -i 's|XRD_REDIR=.*|XRD_REDIR=%s|' ~/vpp-frr/xrootd/.env" % xrd_redir]

for host in list(hosts.keys()):
    runcmd([host], commands)

# Start XRootD Process on all nodes;

In [ ]:
commands = ["cd vpp-frr/xrootd/ && sudo docker compose up -d"]
for host in list(hosts.keys()):
    runcmd([host], commands)

## Extend slice to max (2 weeks)

Extend slice for another two weeks to keep it active

In [ ]:
# Extend slice (if already present)
from datetime import datetime
from datetime import timezone
from datetime import timedelta
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config();

#Set end host to now plus 1 day
end_date = (datetime.now(timezone.utc) + timedelta(days=14)).strftime("%Y-%m-%d %H:%M:%S %z")

try:
    slice = fablib.get_slice(name=slice_name)
    slice.renew(end_date)
except Exception as e:
    print(f"Exception: {e}")

# Check slice end date
try:
    slice = fablib.get_slice(name=slice_name)
    print(f"Lease End (UTC)        : {slice.get_lease_end()}")
       
except Exception as e:
    print(f"Exception: {e}")